In [1]:
import torch
from torch.distributions.laplace import Laplace
import numpy as np

In [2]:
def create_db(num_entries):
    full_db = torch.rand(num_entries) > 0.5
    return full_db

def create_parallel_dbs(full_db):
    parallel_dbs = []
    for i in range(len(full_db)):
        parallel_dbs.append(torch.cat((full_db[0:i], full_db[i+1:])))
    return parallel_dbs

In [3]:
def sum_query(db):
    return db.sum()

In [9]:
def mean_query(db):
    return torch.mean(db.float())

In [5]:
#epsilon for each query. If more than one query it would be divided across all the queries
#the lower the epsilon the more noise would need to be added to ensure leakage is not more than epsilon
epsilon = 0.5

In [15]:
#global differential privacy - laplacian noise or gaussian noise. delta is not used for laplacian noise bu only for gaussian
#if the query is less sensitive we would need to add less amount of noise. The more sensitive the more noise.
def add_laplacian_noise(db, query, query_sensitivity):
    beta = query_sensitivity / epsilon
    #lp = Laplace(torch.tensor([0.]), beta, torch.tensor([1.]))
    #laplacian_noise = lp.sample()
    laplacian_noise = torch.tensor(np.random.laplace(0, beta, 1))
    return sum_query(db) + laplacian_noise

In [16]:
num_entries = 100
db = create_db(num_entries)

true_result = sum_query(db)
private_result = add_laplacian_noise(db, sum_query, 1) #sensitivity of sum query is 1

print(true_result)
print(private_result)

tensor(54)
tensor([53.5087], dtype=torch.float64)


In [18]:
true_result = mean_query(db)
private_result = add_laplacian_noise(db, mean_query, 0.01) #sensitivity of mean query is 1/100

print(true_result)
print(private_result)

tensor(0.5400)
tensor([53.9706], dtype=torch.float64)
